In [1]:
import pandas as pd
import os

folder_path = '/Users/markuswiikjensen/Desktop/Master-Data/NO6_data'  # replace with the path to your folder
dfs = []  # initialize an empty list to store the DataFrames

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    dfs.append(file_path)  # append the DataFrame to the list

In [2]:
len(dfs)

11

In [3]:
dfs

['/Users/markuswiikjensen/Desktop/Master-Data/NO6_data/net_exchange.csv',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO6_data/prices_brent_lng_updated.csv',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO6_data/.DS_Store',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO6_data/da_production_prognosis.csv',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO6_data/electricity_consumption.csv',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO6_data/elspot_prices_dt_interpolated.csv',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO6_data/weather_averaged_preprocessed_NO6.csv',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO6_data/reservoir_levels.csv',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO6_data/prices_osebx_updated.csv',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO6_data/wind_settlement.csv',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO6_data/electricity_production.csv']

In [4]:
df1=pd.read_csv('/Users/markuswiikjensen/Desktop/Master-Data/NO6_data/elspot_prices_dt_interpolated.csv')
df1.set_index('datetime', inplace=True)
df1 = df1[~df1.index.duplicated()]

In [5]:
df1=df1['Tr.heim']
df1 = df1.to_frame('price')

In [6]:
df2=pd.read_csv('/Users/markuswiikjensen/Desktop/Master-Data/NO6_data/electricity_consumption.csv')
df2.set_index('datetime', inplace=True)
df2 = df2[~df2.index.duplicated()]

In [7]:
df2=df2['NO3']
df2.sort_index(ascending=True, inplace=True)
df2 = df2.to_frame('consumption')

In [8]:
df3=pd.read_csv('/Users/markuswiikjensen/Desktop/Master-Data/NO6_data/electricity_production.csv')
df3.set_index('datetime', inplace=True)
df3 = df3[~df3.index.duplicated()]

In [9]:
df3=df3['NO3']
df3.sort_index(ascending=True, inplace=True)
df3 = df3.to_frame('production')

In [10]:
df4=pd.read_csv('/Users/markuswiikjensen/Desktop/Master-Data/NO6_data/da_production_prognosis.csv')
df4.set_index('datetime', inplace=True)
df4 = df4[~df4.index.duplicated()]

In [11]:
df4=df4['NO3']
df4.sort_index(ascending=True, inplace=True)
df4 = df4.to_frame('da_production')

In [12]:
df5=pd.read_csv('/Users/markuswiikjensen/Desktop/Master-Data/NO6_data/net_exchange.csv')
df5.set_index('datetime', inplace=True)
df5 = df5[~df5.index.duplicated()]

In [13]:
df5.sort_index(ascending=True, inplace=True)
filtered_cols = df5.columns[df5.columns.str.contains('NO3')]
df5=df5[filtered_cols]
df5=df5.rename(columns={'NO1_NO6': 'exchange_NO1_NO6', 'NO6_NO4':'exchange_NO6_NO4', 'NO6_NO5':'exchange_NO6_NO5', 'NO6_SE2':'exchange_NO6_SE2'})
df5

,NO1_NO3,NO3_NO4,NO3_NO5,NO3_SE2
datetime,,,,
2019-12-30 00:00:00,-156.0,28.0,21.0,705.0
2019-12-30 01:00:00,-156.0,5.0,143.0,793.0
2019-12-30 02:00:00,-204.0,-80.0,193.0,756.0
2019-12-30 03:00:00,-204.0,-127.0,171.0,723.0
2019-12-30 04:00:00,-211.0,-133.0,158.0,742.0
...,...,...,...,...
2023-04-07 19:00:00,NaN,NaN,NaN,NaN
2023-04-07 20:00:00,NaN,NaN,NaN,NaN
2023-04-07 21:00:00,NaN,NaN,NaN,NaN


In [15]:
df7=pd.read_csv('/Users/markuswiikjensen/Desktop/Master-Data/NO6_data/weather_averaged_preprocessed_NO6.csv')
# Convert the 'date' column to datetime
df7['referenceTime'] = pd.to_datetime(df7['referenceTime'])

# Extract the date part and assign it back to the 'date' column
df7['datetime'] = df7['referenceTime'].dt.date
df7['datetime'] = pd.to_datetime(df7['datetime'])
df7.set_index('datetime', inplace=True)
df7 = df7[~df7.index.duplicated()]
df7=df7.rename(columns={'mean(air_temperature P1D)': 'air_temperature', 'mean(wind_speed P1D)': 'wind_speed','sum(precipitation_amount P1D)':'precipitation_amount' })
df7.drop('referenceTime', axis=1, inplace=True)
df7 = df7.resample('H').ffill()


In [16]:
df8=pd.read_csv('/Users/markuswiikjensen/Desktop/Master-Data/NO6_data/reservoir_levels.csv')
# Convert the 'date' column to datetime
df8['datetime'] = pd.to_datetime(df8['Year/Week'] + '-1', format='%Y/%U-%w')
df8.set_index('datetime', inplace=True)
df8 = df8[~df8.index.duplicated()]
# 1. Convert string dates to datetime objects
start_date = pd.to_datetime('2019-12-29')
end_date = pd.to_datetime('2023-04-01')

# 2. Create new index using pd.date_range()
new_index = pd.date_range(start=start_date, end=end_date, freq='H')

# 3. Reindex DataFrame using new index
df8 = df8.reindex(pd.to_datetime(new_index))

df8 = df8.fillna(method='backfill')
df8 =df8.fillna(method='ffill')
df8=df8[['Capacity_GWh', 'Levels_GWh']]
df8=df8.rename(columns={'Capacity_GWh':'reservoir_capacity', 'Levels_GWh': 'reservoir_levels'})


In [17]:
df9=pd.read_csv('/Users/markuswiikjensen/Desktop/Master-Data/NO6_data/prices_brent_lng_updated.csv')
df9['datetime'] = pd.to_datetime(df9['Date'])
df9.set_index('datetime', inplace=True, drop=True)
df9 = df9[~df9.index.duplicated()]
df9 = df9.resample('H').ffill()
df9=df9[['BZ=F','NG=F']]
df9=df9.rename(columns={'BZ=F':'oil_price','NG=F':'gas_price' })

In [18]:
df10=pd.read_csv('/Users/markuswiikjensen/Desktop/Master-Data/NO6_data/prices_osebx_updated.csv')
df10['datetime'] = pd.to_datetime(df10['Date'])
df10.set_index('datetime', inplace=True, drop=True)
df10 = df10[~df10.index.duplicated()]
df10 = df10.resample('H').ffill()
df10 = df10.rename(columns={'Adj Close': 'osebx_price'})
df10 =df10['osebx_price']
df10 = df10.to_frame('osebx_price')

In [19]:
dfs = [df1, df2, df3, df4, df5, df7, df8, df9, df10]
df=df1.join(df2, on=df1.index, how='left')
df=df.join(df3,  on=df1.index, how='left')
df=df.join(df4,  on=df1.index, how='left')
df=df.join(df5,  on=df1.index, how='left')

In [20]:
df.index = pd.to_datetime(df.index, format='%Y-%m-%d %H:%M:%S')

In [21]:
df=df.join(df7,  on='datetime', how='left')
df=df.join(df8,  on='datetime', how='left')
df=df.join(df9,  on='datetime', how='left')
df=df.join(df10,  on='datetime', how='left')

In [22]:
df

,price,consumption,production,da_production,exchange_NO1_NO5,exchange_NO2_NO5,exchange_NO3_NO5,air_temperature,wind_speed,precipitation_amount,reservoir_capacity,reservoir_levels,oil_price,gas_price,osebx_price
datetime,,,,,,,,,,,,,,,
2020-01-01 00:00:00,31386.0,1981.0,2720.0,2765.0,739.0,42.0,-41.0,3.784574,7.705263,3.432857,87301.0,55923.0,NaN,NaN,NaN
2020-01-01 01:00:00,31336.0,1998.0,3775.0,3624.0,1506.0,386.0,-114.0,3.784574,7.705263,3.432857,87301.0,55923.0,NaN,NaN,NaN
2020-01-01 02:00:00,31139.0,1950.0,3698.0,3520.0,1440.0,341.0,-34.0,3.784574,7.705263,3.432857,87301.0,55923.0,NaN,NaN,NaN
2020-01-01 03:00:00,30853.0,1926.0,3469.0,3339.0,1279.0,287.0,-23.0,3.784574,7.705263,3.432857,87301.0,55923.0,NaN,NaN,NaN
2020-01-01 04:00:00,30301.0,1913.0,3058.0,3031.0,961.0,188.0,-4.0,3.784574,7.705263,3.432857,87301.0,55923.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-17 19:00:00,117379.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-04-17 20:00:00,117254.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-04-17 21:00:00,117014.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df=df.fillna(method='backfill')
df=df.dropna()
df

,price,consumption,production,da_production,exchange_NO1_NO5,exchange_NO2_NO5,exchange_NO3_NO5,air_temperature,wind_speed,precipitation_amount,reservoir_capacity,reservoir_levels,oil_price,gas_price,osebx_price
datetime,,,,,,,,,,,,,,,
2020-01-01 00:00:00,31386.0,1981.0,2720.0,2765.0,739.0,42.0,-41.0,3.784574,7.705263,3.432857,87301.0,55923.0,582.464933,18.656462,941.270020
2020-01-01 01:00:00,31336.0,1998.0,3775.0,3624.0,1506.0,386.0,-114.0,3.784574,7.705263,3.432857,87301.0,55923.0,582.464933,18.656462,941.270020
2020-01-01 02:00:00,31139.0,1950.0,3698.0,3520.0,1440.0,341.0,-34.0,3.784574,7.705263,3.432857,87301.0,55923.0,582.464933,18.656462,941.270020
2020-01-01 03:00:00,30853.0,1926.0,3469.0,3339.0,1279.0,287.0,-23.0,3.784574,7.705263,3.432857,87301.0,55923.0,582.464933,18.656462,941.270020
2020-01-01 04:00:00,30301.0,1913.0,3058.0,3031.0,961.0,188.0,-4.0,3.784574,7.705263,3.432857,87301.0,55923.0,582.464933,18.656462,941.270020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-29 20:00:00,144605.0,2166.0,5460.0,5579.0,2710.0,547.0,39.0,-2.373846,3.070732,4.552817,87301.0,53981.0,815.527912,20.742414,1168.589966
2023-03-29 21:00:00,125662.0,2203.0,5588.0,5522.0,2840.0,472.0,64.0,-2.373846,3.070732,4.552817,87301.0,53981.0,815.527912,20.742414,1168.589966
2023-03-29 22:00:00,116965.0,2220.0,5520.0,5433.0,2810.0,470.0,6.0,-2.373846,3.070732,4.552817,87301.0,53981.0,815.527912,20.742414,1168.589966


In [24]:
df.to_csv('NO6_preprocessed')